# Day 12: Rain Risk  -  *WORK IN PROGRESS*
https://adventofcode.com/2020/day/12

In [1]:
inputLines = lines <$> readFile "input/day12.txt"

In [2]:
testInput = [ "F10"
            , "N3"
            , "F7"
            , "R90"
            , "F11" ]

In [3]:
import Text.Regex.PCRE  -- install with 'stack install regex-pcre'

In [4]:
import Linear.V2  -- install with 'stack install linear'
import Linear.Matrix

In [5]:
newtype Position = Position (V2 Int) deriving (Show)

In [6]:
newtype Direction = Direction (V2 Int) deriving (Show)

In [7]:
go (Position pos) (Direction v) n = Position $ pos + V2 n n * v

In [8]:
left :: Direction -> Direction
left (Direction v) = Direction $ rotationMatrix !* v
    where rotationMatrix = transpose $ V2 (V2 0 1) (V2 (-1) 0)

In [9]:
north = Direction $ V2 0 1

In [10]:
[_, west, south, east] = take 4 . iterate left $ north

In [11]:
(north, east, south, west)

(Direction (V2 0 1),Direction (V2 1 0),Direction (V2 0 (-1)),Direction (V2 (-1) 0))

In [12]:
right = left . left . left

In [13]:
data ShipState = ShipState Position Direction deriving (Show)

In [14]:
parseInstruction :: String -> (ShipState -> ShipState)
parseInstruction instruction = case head letter of
    'L' -> rotate left
    'R' -> rotate right
    'F' -> goForward
    fixedDirection -> goFixedDirection . parseDirection $ fixedDirection  -- one of NSEW

    where
        [_, letter, argumentStr] = head (instruction =~ "([NSEWLRF])([0-9]+)" :: [[String]])
        
        argument = read argumentStr :: Int
        
        rotate f (ShipState position direction) =
            ShipState position (iterate f direction !! numRotations)
            where numRotations = argument `div` 90
        
        goForward (ShipState position direction) =
            ShipState (go position direction argument) direction
        
        goFixedDirection fixedDirection (ShipState position direction) =
            ShipState (go position fixedDirection argument) direction
        
        parseDirection 'N' = north
        parseDirection 'S' = south
        parseDirection 'E' = east
        parseDirection 'W' = west

In [15]:
parseInstructions = map parseInstruction

In [16]:
executeInstructions :: ShipState -> [ShipState -> ShipState] -> ShipState
executeInstructions = foldl (flip ($))

In [17]:
solution1 :: [String] -> Int
solution1 input = abs x + abs y
    where
        (ShipState (Position (V2 x y)) d) = executeInstructions (ShipState (Position (V2 0 0)) east) . parseInstructions $ input

In [18]:
solution1 testInput

25

In [19]:
solution1 <$> inputLines

1221